<a href="https://colab.research.google.com/github/fitocuan/CE888_2020/blob/master/my_recommender.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
import numpy as np
from IPython.display import Image
np.set_printoptions(precision = 3)

In [0]:
df = pd.read_csv('jester-data-1.csv')
print(df.head())
print(df.shape)

    74  -7.82   8.79  -9.66  -8.16  ...  99.22  -5.63  99.23  99.24  99.25
0  100   4.08  -0.29   6.36   4.37  ...  -2.14   3.06   0.34  -4.32   1.07
1   49  99.00  99.00  99.00  99.00  ...  99.00  99.00  99.00  99.00  99.00
2   48  99.00   8.35  99.00  99.00  ...  99.00  99.00  99.00  99.00  99.00
3   91   8.50   4.61  -4.17  -5.39  ...   1.55   3.11   6.55   1.80   1.60
4  100  -6.17  -3.54   0.44  -8.50  ...  -3.35   0.05  -9.08  -5.05  -3.45

[5 rows x 101 columns]
(24982, 101)


In [0]:

arr = df.drop(df.columns[0], axis=1).values
print(arr)

[[ 4.08 -0.29  6.36 ...  0.34 -4.32  1.07]
 [99.   99.   99.   ... 99.   99.   99.  ]
 [99.    8.35 99.   ... 99.   99.   99.  ]
 ...
 [99.   99.   99.   ... 99.   99.   99.  ]
 [99.   99.   99.   ... 99.   99.   99.  ]
 [ 2.43  2.67 -3.98 ... 99.   99.   99.  ]]


In [0]:
def replace(orig, percentage=0.1):
  """
  Replaces 'percentage'% of the original values in 'orig' with 99's
  :param orig: original data array
  :param percentage: percentage of values to replace (0<percentage<1)
  """
  new_data = orig.copy()
  rated = np.where(arr!=99)
  n_rated = len(rated[0])
  idx = np.random.choice(n_rated, size=int(percentage*n_rated), replace=False)
  new_data[rated[0][idx], rated[1][idx]] = 99
  return new_data, (rated[0][idx], rated[1][idx])

In [0]:

arr_new, idx = replace(arr, 0.1)

In [0]:

rated = np.where(arr!=99)
rated_new = np.where(arr_new!=99)
print(len(rated[0]), rated[1].shape)




1810381 (1810381,)


In [0]:
n_latent_factors = 2

user_ratings = arr
# Initialise as random values
latent_user_preferences = np.random.random((user_ratings.shape[0], n_latent_factors))
latent_item_features = np.random.random((user_ratings.shape[1], n_latent_factors))

print(user_ratings)


[[ 4.08 -0.29  6.36 ...  0.34 -4.32  1.07]
 [99.   99.   99.   ... 99.   99.   99.  ]
 [99.    8.35 99.   ... 99.   99.   99.  ]
 ...
 [99.   99.   99.   ... 99.   99.   99.  ]
 [99.   99.   99.   ... 99.   99.   99.  ]
 [ 2.43  2.67 -3.98 ... 99.   99.   99.  ]]


In [0]:

def predict_rating(user_id, item_id):
    """ Predict a rating given a user_id and an item_id.
    """
    user_preference = latent_user_preferences[user_id]
    item_preference = latent_item_features[item_id]
    return user_preference.dot(item_preference)

def train(user_id, item_id, rating, alpha=0.0001):
    
    #print item_id
    prediction_rating = predict_rating(user_id, item_id)
    err =  prediction_rating - rating
    #print err
    user_pref_values = latent_user_preferences[user_id][:]

    latent_user_preferences[user_id] -= alpha * err * latent_item_features[item_id]
    latent_item_features[item_id] -= alpha * err * user_pref_values

    
    return err
    

def sgd(iterations=1000):
    """ Iterate over all users and all items and train for 
        a certain number of iterations
    """
    for iteration in range(iterations):
        error = []
        for user_id in range(latent_user_preferences.shape[0]):
            for item_id in range(latent_item_features.shape[0]):
                rating = user_ratings[user_id][item_id]
                if rating != 99:
                    err = train(user_id, item_id, rating)
                    error.append(err)
        mse = (np.array(error) ** 2).mean()   
        if (iteration % 10) == 0:
            print(mse)


In [0]:
sgd() 

In [149]:
predictions = latent_user_preferences.dot(latent_item_features.T)

values = [zip(user_ratings[i], predictions[i]) for i in range(predictions.shape[0])]
comparison_data = pd.DataFrame(values)
comparison_data

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99
0,"(4.08, 0.8639874734844573)","(-0.29, 0.35129838344736547)","(6.36, 0.37788720558277367)","(4.37, -0.8185604981042067)","(-2.38, 0.4870160497082758)","(-9.66, 1.4361534916417988)","(-0.73, -0.16003706364496728)","(-5.34, -0.4432057844834755)","(8.88, -0.19888916386212882)","(9.22, 1.1498399985677465)","(6.75, 1.571732661649388)","(8.64, 1.3548043813059267)","(4.42, -1.139541053520513)","(7.43, 1.3242919315738524)","(4.56, -1.1248507537238552)","(-0.97, -2.111458852734112)","(4.66, -0.660935793827873)","(-0.68, -0.30350885440040165)","(3.3, 0.33393556487724374)","(-1.21, -0.5022354731270446)","(0.87, 1.8390140815679341)","(8.64, 0.9540905178273674)","(8.35, 0.35964872372071444)","(9.17, -0.9878885825039498)","(0.05, 0.570139261750941)","(7.57, 1.2357482901916825)","(4.71, 2.55157015710347)","(0.87, 1.3376577653210433)","(-0.39, 2.2991444897326567)","(6.99, -0.05091591029455565)","(6.5, 1.9464375793185047)","(-0.92, 2.489652124911404)","(7.14, -0.7238925862261054)","(9.03, 0.9265812062442205)","(-1.8, 2.4152087818853003)","(0.73, 2.6085495128898413)","(7.09, -0.7673580835989238)","(3.4, 1.1964883167208382)","(-0.87, 1.1223483454033132)","(7.91, 1.0440073035747026)",...,"(-6.7, 2.1295882716595957)","(-3.35, 2.3942491199975424)","(-9.03, 0.4107461838142741)","(4.47, -0.25666458138576154)","(4.08, 1.9593319817674206)","(-3.83, 2.12614485856112)","(8.74, -0.3418550426853512)","(1.12, 2.1843626657450406)","(0.78, 2.1549156659409774)","(7.52, 0.6962051721913175)","(-5.0, -0.15637637702271637)","(2.77, 2.0921697705440185)","(8.3, 1.0124711194604241)","(7.77, -0.7934017362038983)","(7.33, 0.09209039449208324)","(6.21, 1.8927710257951427)","(7.72, 0.7774679984675777)","(8.98, 1.4577953412209683)","(8.64, 0.3921325058669684)","(8.2, 1.073012294995289)","(3.93, 1.4410946205473343)","(4.85, 0.9338018131705192)","(4.85, 1.5748645908937877)","(6.07, 0.6866836413209266)","(8.98, 0.9238781634012179)","(4.51, 0.4024741807486167)","(-0.05, 1.5233389652269878)","(3.69, 1.6301029051760385)","(4.56, 2.6433891139406365)","(0.58, 0.671545576471788)","(2.82, 1.6462718137396666)","(-4.95, 0.9935773685920787)","(-0.29, 1.9975202192379244)","(7.86, 1.036781078378797)","(-0.19, 0.9669855229554446)","(-2.14, 1.3205401350436392)","(3.06, 1.4109092342048424)","(0.34, 0.9128018804687775)","(-4.32, 0.2645556302512925)","(1.07, 1.301896966236498)"
1,"(99.0, 1.552409927691845)","(99.0, 0.6519472207643406)","(99.0, 0.7285887531000639)","(99.0, -1.5342751483716073)","(9.03, 0.8667180800151061)","(9.27, 2.665561735323018)","(9.03, -0.33790787460963784)","(9.27, -0.8125120599027127)","(99.0, -0.36020975626973956)","(99.0, 2.102023472723472)","(7.33, 2.874214547373553)","(7.57, 2.4979941163580084)","(9.37, -2.1228186553881185)","(6.17, 2.435586528328434)","(-6.36, -2.116899503943356)","(-6.89, -3.986841655729695)","(-7.86, -1.2415000499424347)","(9.03, -0.5900757726747796)","(9.03, 0.5733365859430797)","(9.03, -0.9166133667128417)","(7.28, 3.3874254857686896)","(99.0, 1.739275418944478)","(8.25, 0.6157444603140407)","(99.0, -1.8672750698978886)","(99.0, 1.053410180587694)","(7.48, 2.2991423470256103)","(7.28, 4.7177934299716675)","(7.28, 2.503723159825496)","(8.93, 4.322515179007669)","(99.0, -0.09667798283472846)","(6.17, 3.564702449177358)","(7.28, 4.638475859293405)","(99.0, -1.3880487318622103)","(99.0, 1.7235617233125449)","(8.98, 4.487361607445173)","(7.33, 4.8582561629724506)","(99.0, -1.4466794099117921)","(6.17, 2.199558310321524)","(9.08, 2.058130310103346)","(7.33, 1.900573122689922)",...,"(6.46, 3.942271439228688)","(7.28, 4.474643775404736)","(99.0, 0.7871091672432178)","(99.0, -0.5062307115501098)","(7.04, 3.63221731977313)","(7.28, 3.974015956501581)","(99.0, -0.6783882587286971)","(7.28, 4.082122202559188)","(8.25, 4.01